# Move Bounds

Notebook on moving bounds for systems, while also calculating the the $\sigma$s

In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg
import tvsclib.utils as utils

from tvsclib.transformations.output_normal import OutputNormal
from tvsclib.transformations.input_normal import InputNormal
from tvsclib.transformations.reduction import Reduction

In [ ]:
matrix = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
#matrix = np.random.rand(12,12)
dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)

system = MixedSystem(S)
utils.show_system(system)

First work on the casual part:

Require a output normal system, start with the lowest index and work up, while transforming to a input normal system

# Move Right/left

In [ ]:
def transform(stages_causal,stages_anticausal,direction):
    k = len(stages_causal)
    sigmas_causal = []
    sigmas_anticausal = []
    for i in range(1,len(stages_causal)):#loop over inices of causal states
        if direction == "l":
            #move left:
            b = stages_causal[i-1].B_matrix[:,-1:]
            U,s_l,Vt= np.linalg.svd(np.hstack([stages_causal[i-1].A_matrix,stages_causal[i-1].B_matrix[:,:-1]]),full_matrices=False)
            Us=U*s_l

            stages_l = [
                Stage(Vt[:,:stages_causal[i-1].A_matrix.shape[1]],Vt[:,stages_causal[i-1].A_matrix.shape[1]:],\
                        stages_causal[i-1].C_matrix,stages_causal[i-1].D_matrix[:,:-1]),
                Stage(stages_causal[i].A_matrix@Us,np.hstack((stages_causal[i].A_matrix@b,stages_causal[i].B_matrix)),\
                        stages_causal[i].C_matrix@Us,np.hstack((stages_causal[i].C_matrix@b,stages_causal[i].D_matrix)))
            ]

        if direction =="r"or direction=="n":
            #no move-> only make R_k input normal
            U,s,Vt= np.linalg.svd(np.hstack([stages_causal[i-1].A_matrix,stages_causal[i-1].B_matrix]),full_matrices=False)
            Us=U*s

            stages_n=[
                Stage(Vt[:,:stages_causal[i-1].A_matrix.shape[1]],Vt[:,stages_causal[i-1].A_matrix.shape[1]:],\
                        stages_causal[i-1].C_matrix,stages_causal[i-1].D_matrix),
                Stage(stages_causal[i].A_matrix@Us,stages_causal[i].B_matrix,\
                        stages_causal[i].C_matrix@Us,stages_causal[i].D_matrix)
            ]

        #move right -> base on non move
        if direction =="r":
            b = stages_n[1].B_matrix[:,:1]
            d = stages_n[1].D_matrix[:,:1]
            #d_add = np.zeros((stages_n[0].D_matrix.shape[0],1))
            d_add = stages_anticausal[i-1].C_matrix@stages_anticausal[i].B_matrix[:,:1]


            U,s_r,Vt= np.linalg.svd(np.block([[stages_n[1].A_matrix,b],
                                              [stages_n[1].C_matrix,d]]),full_matrices=False)
            Us=U*s_r
            stages_r=[
                 #Here the A and B are more complicated as we have to stack them
                Stage(Vt@(np.vstack([stages_n[0].A_matrix,np.zeros((1,stages_n[0].A_matrix.shape[1]))])),
                      Vt@(np.block([[stages_n[0].B_matrix,np.zeros((stages_n[0].B_matrix.shape[0],1))],
                                   [np.zeros(stages_n[0].B_matrix.shape[1]),np.eye(1)]])),
                      stages_n[0].C_matrix,np.hstack([stages_n[0].D_matrix,d_add])),

               Stage(Us[:stages_n[1].A_matrix.shape[0],:],stages_n[1].B_matrix[:,1:],\
                     Us[stages_n[1].A_matrix.shape[0]:,:],stages_n[1].D_matrix[:,1:])
            ]



        # Now calculate the anticausla part:
        #move left
        if direction =="l":
            b = stages_anticausal[i-1].B_matrix[:,-1:]
            d = stages_causal[i-1].D_matrix[:,-1:]
            d_add = np.zeros((stages_anticausal[i].D_matrix.shape[0],1))
            #d_add = stages_causal[i].C_matrix@stages_causal[i-1].B_matrix[:,-1:]
            
            U,s_al,Vt= np.linalg.svd(np.block([[b,stages_anticausal[i-1].A_matrix],
                                               [d,stages_anticausal[i-1].C_matrix]]),full_matrices=False)
            sVt=s_al.reshape(-1,1)*Vt
            stages_anti_l=[
                Stage(U[:stages_anticausal[i-1].A_matrix.shape[0],:],stages_anticausal[i-1].B_matrix[:,:-1],\
                      U[stages_anticausal[i-1].A_matrix.shape[0]:,:],stages_anticausal[i-1].D_matrix[:,:-1]),
                 #Here the A and B are more complicated as we have to stack them
                Stage(sVt@(np.vstack([np.zeros((1,stages_anticausal[i].A_matrix.shape[1])),stages_anticausal[i].A_matrix])),
                      sVt@(np.block([[np.eye(1),np.zeros(stages_anticausal[i].B_matrix.shape[1])],
                                     [np.zeros((stages_anticausal[i].B_matrix.shape[0],1)),stages_anticausal[i].B_matrix]])),
                      stages_anticausal[i].C_matrix,np.hstack([d_add,stages_anticausal[i].D_matrix]))
            ]


        if direction =="r"or direction=="n":
            #no move-> only make R_k input normal
            U,s_a,Vt= np.linalg.svd(np.vstack([stages_anticausal[i-1].A_matrix,stages_anticausal[i-1].C_matrix]),full_matrices=False)
            sVt=s_a.reshape(-1,1)*Vt

            stages_anti_n=[
                Stage(U[:stages_anticausal[i-1].A_matrix.shape[0],:],stages_anticausal[i-1].B_matrix,\
                      U[stages_anticausal[i-1].A_matrix.shape[0]:,:],stages_anticausal[i-1].D_matrix),
                Stage(sVt@stages_anticausal[i].A_matrix,sVt@stages_anticausal[i].B_matrix,\
                      stages_anticausal[i].C_matrix,stages_anticausal[i].D_matrix)
            ]


        if direction == "r":
            #move right: -> base on non move
            b = stages_anti_n[1].B_matrix[:,:1]
            U,s_ar,Vt= np.linalg.svd(np.hstack([stages_anti_n[1].A_matrix,stages_anti_n[1].B_matrix[:,1:]]),full_matrices=False)
            sVt=s_ar.reshape(-1,1)*Vt

            stages_anti_r = [
                Stage(stages_anti_n[0].A_matrix@U,np.hstack((stages_anti_n[0].B_matrix,stages_anti_n[0].A_matrix@b)),\
                      stages_anti_n[0].C_matrix@U,np.hstack((np.zeros((stages_anti_n[0].D_matrix.shape[0],1)),stages_anti_n[0].D_matrix))),
                #D is here 0, instead we have d_add at the causal system. Insted we could use stages_anti_n[0].C_matrix@b
                Stage(sVt[:,:stages_anti_n[1].A_matrix.shape[1]],sVt[:,stages_anti_n[1].A_matrix.shape[1]:],\
                        stages_anti_n[1].C_matrix,stages_anti_n[1].D_matrix[:,:-1])
            ]


        if direction == "l":
            stages_causal[i-1]= stages_l[0]
            stages_causal[i]= stages_l[1]
            stages_anticausal[i-1]= stages_anti_l[0]
            stages_anticausal[i]= stages_anti_l[1]
            sigmas_causal.append(s_l)
            sigmas_anticausal.append(s_al)
        if direction == "n":
            stages_causal[i-1]= stages_n[0]
            stages_causal[i]= stages_n[1]
            stages_anticausal[i-1]= stages_anti_n[0]
            stages_anticausal[i]= stages_anti_n[1]
            sigmas_causal.append(s)
            sigmas_anticausal.append(s_a)
        if direction == "r":
            stages_causal[i-1]= stages_r[0]
            stages_causal[i]= stages_r[1]
            stages_anticausal[i-1]= stages_anti_r[0]
            stages_anticausal[i]= stages_anti_r[1]
            sigmas_causal.append(s_r)
            sigmas_anticausal.append(s_ar)
    return sigmas_causal,sigmas_anticausal


In [ ]:
sys_move_causal = system.causal_system.copy()
sys_move_anticausal = system.anticausal_system.copy()

sys_move_causal = OutputNormal().apply(sys_move_causal)
sys_move_anticausal = InputNormal().apply(sys_move_anticausal)

sys_move =MixedSystem(causal_system=sys_move_causal,anticausal_system=sys_move_anticausal)

sigmas_causal,sigmas_anticausal=transform(sys_move.causal_system.stages,sys_move.anticausal_system.stages,"l")
utils.show_system(sys_move)
A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

print("causal:")
for k in range(1,len(system.causal_system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_causal[k-1])]=sigmas_causal[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])
    
print("anticausal:")
for k in range(0,len(system.causal_system.stages)-1):
    sigmas_r = np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_anticausal[k])]=sigmas_anticausal[k]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas_anticausal[k])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
sys_move_causal = system.causal_system.copy()
sys_move_anticausal = system.anticausal_system.copy()

sys_move_causal = OutputNormal().apply(sys_move_causal)
sys_move_anticausal = InputNormal().apply(sys_move_anticausal)

sys_move =MixedSystem(causal_system=sys_move_causal,anticausal_system=sys_move_anticausal)

sigmas_causal,sigmas_anticausal=transform(sys_move.causal_system.stages,sys_move.anticausal_system.stages,"n")
utils.show_system(sys_move)
A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

print("causal:")
for k in range(1,len(system.causal_system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_causal[k-1])]=sigmas_causal[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])
    
print("anticausal:")
for k in range(0,len(system.causal_system.stages)-1):
    sigmas_r = np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_anticausal[k])]=sigmas_anticausal[k]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas_anticausal[k])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
sys_move_causal = system.causal_system.copy()
sys_move_anticausal = system.anticausal_system.copy()

sys_move_causal = OutputNormal().apply(sys_move_causal)
sys_move_anticausal = InputNormal().apply(sys_move_anticausal)

sys_move =MixedSystem(causal_system=sys_move_causal,anticausal_system=sys_move_anticausal)

sigmas_causal,sigmas_anticausal=transform(sys_move.causal_system.stages,sys_move.anticausal_system.stages,"r")
utils.show_system(sys_move)
A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

print("causal:")
for k in range(1,len(system.causal_system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_causal[k-1])]=sigmas_causal[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])
    
print("anticausal:")
for k in range(0,len(system.causal_system.stages)-1):
    sigmas_r = np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_anticausal[k])]=sigmas_anticausal[k]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas_anticausal[k])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

# Move up/down

In [ ]:
def transform_updown(stages_causal,stages_anticausal,direction):
    k = len(stages_causal)
    sigmas_causal = []
    sigmas_anticausal = []
    for i in range(k-1, 0,-1):
        print("i:",i)
        if direction == "d":
            #move down:
            c = stages_causal[i].C_matrix[:1,:]
            U,s_d,Vt= np.linalg.svd(np.vstack([stages_causal[i].A_matrix,stages_causal[i].C_matrix[1:,:]]),full_matrices=False)
            sVt=s_d.reshape(-1,1)*Vt

            stages_d = [
                Stage(sVt@stages_causal[i-1].A_matrix,sVt@stages_causal[i-1].B_matrix,\
                    np.vstack([stages_causal[i-1].C_matrix,c@stages_causal[i-1].A_matrix]),
                      np.vstack([stages_causal[i-1].D_matrix,c@stages_causal[i-1].B_matrix])),
                Stage(U[:stages_causal[i].A_matrix.shape[0],:],stages_causal[i].B_matrix,\
                      U[stages_causal[i].A_matrix.shape[0]:,:],stages_causal[i].D_matrix[1:,:])
            ]


        if direction =="u"or direction=="n":
            #no move-> only make O_k normal
            U,s,Vt= np.linalg.svd(np.vstack([stages_causal[i].A_matrix,stages_causal[i].C_matrix]),full_matrices=False)
            sVt=s.reshape(-1,1)*Vt

            stages_n=[
                Stage(sVt@stages_causal[i-1].A_matrix,sVt@stages_causal[i-1].B_matrix,\
                    stages_causal[i-1].C_matrix,stages_causal[i-1].D_matrix),
                Stage(U[:stages_causal[i].A_matrix.shape[0],:],stages_causal[i].B_matrix,\
                      U[stages_causal[i].A_matrix.shape[0]:,:],stages_causal[i].D_matrix)
            ]
            
        if direction == "u":
            c = stages_n[0].C_matrix[-1:,:]
            d = stages_n[0].D_matrix[-1:,:]
            #d_add = np.zeros((1,stages_n[1].D_matrix.shape[1]))
            d_add = stages_anticausal[i-1].C_matrix[-1:,:]@stages_anticausal[i].B_matrix


            U,s_u,Vt= np.linalg.svd(np.block([[c,d],
                                              [stages_n[0].A_matrix,stages_n[0].B_matrix]]),full_matrices=False)
            sVt=s_u.reshape(-1,1)*Vt
            stages_u=[
               Stage(sVt[:,:stages_n[0].A_matrix.shape[1]],sVt[:,stages_n[0].A_matrix.shape[1]:],\
                     stages_n[0].C_matrix[:-1,:],stages_n[0].D_matrix[:-1,:]),
                 #Here the A and C are more complicated as we have to stack them
                Stage(np.hstack([np.zeros((stages_n[1].A_matrix.shape[0],1)),stages_n[1].A_matrix])@U,
                      stages_n[1].B_matrix,\
                      np.block([[np.eye(1),np.zeros((1,stages_n[1].C_matrix.shape[1]))],
                                [np.zeros((stages_n[1].C_matrix.shape[0],1)),stages_n[1].C_matrix]])@U,
                      np.vstack([d_add,stages_n[1].D_matrix]))
            ]

        # Now calculate the anticausal part:
        if direction =="d":
            c = stages_anticausal[i].C_matrix[:1,:]
            d = stages_causal[i].D_matrix[:1,:]
            d_add = np.zeros((1,stages_anticausal[i-1].D_matrix.shape[1]))
            #d_add = stages_causal[i].C_matrix@stages_causal[i-1].B_matrix[:,-1:]
            
            U,s_ad,Vt= np.linalg.svd(np.block([[stages_anticausal[i].A_matrix,stages_anticausal[i].B_matrix],
                                               [c,d]]),full_matrices=False)
            Us=U*s_ad
            stages_anti_d=[
                 #Here the A and B are more complicated as we have to stack them
                Stage((np.hstack([stages_anticausal[i-1].A_matrix,np.zeros((stages_anticausal[i-1].A_matrix.shape[0],1))]))@Us,
                      stages_anticausal[i-1].B_matrix,
                      np.block([[stages_anticausal[i-1].C_matrix,np.zeros((stages_anticausal[i-1].C_matrix.shape[0],1))],
                                [np.zeros((1,stages_anticausal[i-1].C_matrix.shape[1])),np.eye(1)]])@Us,
                      np.vstack([stages_anticausal[i-1].D_matrix,d_add])),
                Stage(Vt[:,:stages_anticausal[i].A_matrix.shape[1]],Vt[:,stages_anticausal[i].A_matrix.shape[1]:],\
                      stages_anticausal[i].C_matrix[1:,:],stages_anticausal[i].D_matrix[1:,:])
            ]        

        if direction =="u"or direction=="n":
            #no move-> only make R_k input normal
            U,s_a,Vt= np.linalg.svd(np.hstack([stages_anticausal[i].A_matrix,stages_anticausal[i].B_matrix]),full_matrices=False)
            Us=U*s_a

            stages_anti_n=[
                Stage(stages_anticausal[i-1].A_matrix@Us,stages_anticausal[i-1].B_matrix,\
                      stages_anticausal[i-1].C_matrix@Us,stages_anticausal[i-1].D_matrix),
                Stage(Vt[:,:stages_anticausal[i].A_matrix.shape[1]],Vt[:,stages_anticausal[i].A_matrix.shape[1]:],\
                      stages_anticausal[i].C_matrix,stages_anticausal[i].D_matrix)
            ]

        if direction == "u":
            #move right: -> base on non move
            c = stages_anti_n[0].C_matrix[-1:,:]
            U,s_au,Vt= np.linalg.svd(np.vstack([stages_anti_n[0].A_matrix,stages_anti_n[0].C_matrix[:-1,:]]),full_matrices=False)
            Us=U*s_au

            stages_anti_u = [
                #D is here 0, instead we have d_add at the causal system. Insted we could use stages_anti_n[0].C_matrix@b
                Stage(Us[:stages_anti_n[0].A_matrix.shape[0],:],stages_anti_n[0].B_matrix,\
                      Us[stages_anti_n[0].A_matrix.shape[0]:,:],stages_anti_n[0].D_matrix[:-1,:]),
                Stage(Vt@stages_anti_n[1].A_matrix,Vt@stages_anti_n[1].B_matrix,\
                      np.vstack((c@stages_anti_n[1].A_matrix,stages_anti_n[1].C_matrix)),\
                      np.vstack((np.zeros((1,stages_anti_n[1].D_matrix.shape[1])),stages_anti_n[1].D_matrix)))
            ]
            
            
        if direction == "d":
            stages_causal[i-1]= stages_d[0]
            stages_causal[i]= stages_d[1]
            stages_anticausal[i-1]= stages_anti_d[0]
            stages_anticausal[i]= stages_anti_d[1]
            sigmas_causal.append(s_d)
            sigmas_anticausal.append(s_ad)
        if direction == "n":
            stages_causal[i-1]= stages_n[0]
            stages_causal[i]= stages_n[1]
            stages_anticausal[i-1]= stages_anti_n[0]
            stages_anticausal[i]= stages_anti_n[1]
            sigmas_causal.append(s)
            sigmas_anticausal.append(s_a)
        if direction == "u":
            stages_causal[i-1]= stages_u[0]
            stages_causal[i]= stages_u[1]
            stages_anticausal[i-1]= stages_anti_u[0]
            stages_anticausal[i]= stages_anti_u[1]
            sigmas_causal.append(s_u)
            sigmas_anticausal.append(s_au)
    sigmas_causal.reverse()
    sigmas_anticausal.reverse()
    return sigmas_causal,sigmas_anticausal
            

In [ ]:
sys_move_causal = system.causal_system.copy()
sys_move_anticausal = system.anticausal_system.copy()

sys_move_causal = InputNormal().apply(sys_move_causal)
sys_move_anticausal = OutputNormal().apply(sys_move_anticausal)

sys_move =MixedSystem(causal_system=sys_move_causal,anticausal_system=sys_move_anticausal)

sigmas_causal,sigmas_anticausal=transform_updown(sys_move.causal_system.stages,
                                                 sys_move.anticausal_system.stages,"d")
utils.check_dims(sys_move.causal_system)
utils.show_system(sys_move)
A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

print("causal:")
for k in range(1,len(system.causal_system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_causal[k-1])]=sigmas_causal[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])
    
print("anticausal:")
for k in range(0,len(system.causal_system.stages)-1):
    sigmas_r = np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_anticausal[k])]=sigmas_anticausal[k]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas_anticausal[k])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
sys_move_causal = system.causal_system.copy()
sys_move_anticausal = system.anticausal_system.copy()

sys_move_causal = InputNormal().apply(sys_move_causal)
sys_move_anticausal = OutputNormal().apply(sys_move_anticausal)

sys_move =MixedSystem(causal_system=sys_move_causal,anticausal_system=sys_move_anticausal)

sigmas_causal,sigmas_anticausal=transform_updown(sys_move.causal_system.stages,
                                                 sys_move.anticausal_system.stages,"n")
utils.check_dims(sys_move.causal_system)
utils.show_system(sys_move)
A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

print("causal:")
for k in range(1,len(system.causal_system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_causal[k-1])]=sigmas_causal[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])
    
print("anticausal:")
for k in range(0,len(system.causal_system.stages)-1):
    sigmas_r = np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_anticausal[k])]=sigmas_anticausal[k]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas_anticausal[k])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
sys_move_causal = system.causal_system.copy()
sys_move_anticausal = system.anticausal_system.copy()

sys_move_causal = InputNormal().apply(sys_move_causal)
sys_move_anticausal = OutputNormal().apply(sys_move_anticausal)

sys_move =MixedSystem(causal_system=sys_move_causal,anticausal_system=sys_move_anticausal)

sigmas_causal,sigmas_anticausal=transform_updown(sys_move.causal_system.stages,
                                                 sys_move.anticausal_system.stages,"u")
utils.check_dims(sys_move.causal_system)
utils.show_system(sys_move)
A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

print("causal:")
for k in range(1,len(system.causal_system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_causal[k-1])]=sigmas_causal[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])
    
print("anticausal:")
for k in range(0,len(system.causal_system.stages)-1):
    sigmas_r = np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_anticausal[k])]=sigmas_anticausal[k]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas_anticausal[k])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

# Combine movements

In [ ]:
sys_move_causal = system.causal_system.copy()
sys_move_anticausal = system.anticausal_system.copy()

sys_move_causal = InputNormal().apply(sys_move_causal)
sys_move_anticausal = OutputNormal().apply(sys_move_anticausal)

sys_move =MixedSystem(causal_system=sys_move_causal,anticausal_system=sys_move_anticausal)

sigmas_causal,sigmas_anticausal=transform_updown(sys_move.causal_system.stages,
                                                 sys_move.anticausal_system.stages,"d")

sigmas_causal,sigmas_anticausal=transform(sys_move.causal_system.stages,
                                                 sys_move.anticausal_system.stages,"l")
utils.check_dims(sys_move.causal_system)
utils.show_system(sys_move)
A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

print("causal:")
for k in range(1,len(system.causal_system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_causal[k-1])]=sigmas_causal[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])
    
print("anticausal:")
for k in range(0,len(system.causal_system.stages)-1):
    sigmas_r = np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas_anticausal[k])]=sigmas_anticausal[k]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas_anticausal[k])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
import move

In [ ]:
#matrix = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
matrix = np.random.rand(8*6,8*6)
dims_in =  np.array([8]*6)
dims_out = np.array([8]*6)
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)

system_rand = MixedSystem(S)

def cost(s,s_a):
    c =np.sum(s)+np.sum(s_a)
    return c
  
    
def cost_scaled(s,s_a):
    c =np.sum(s/np.max(s))+np.sum(s_a/np.max(s_a))
    return c

sys_move,input_dims,output_dims = move.move(system_rand,7,cost_scaled)
utils.show_system(sys_move)

print(input_dims)
print(output_dims)

In [ ]:
utils.check_dims(sys_move)

In [ ]:
for i in range(input_dims.shape[0]):
    plt.plot(np.sum(input_dims[:i+1,:],axis=0),"+-")

In [ ]:
for i in range(output_dims.shape[0]):
    plt.plot(np.sum(output_dims[:i+1,:],axis=0),"+-")